# Live Plotting : Theano, Bokeh and iPython/Jupyter

http://blocks.readthedocs.org/en/latest/plotting.html

Producing nice plots from blocks requires 'blocks-extra', which happens to already be included in the requirements.txt file in this repo : 

In [ ]:
# . env/bin/activate
# pip install -r requirements.txt 

And, so that we can do dynamic plotting, start the ``bokeh-server`` on the command line :

In [ ]:
# bokeh-server

Now that your plotting server is up and running, start your main loop and pass the Plot extension. Consider this example of fitting the function $f(x)=x^a$ to $f(x)=x^2$.

In [ ]:
import theano
a = theano.shared(3.)
a.name = 'a'
x = theano.tensor.scalar('data')
cost = abs(x ** 2 - x ** a)
cost.name = 'cost'

We train on a 150 random points in [0,1] :

In [ ]:
import numpy
from fuel.streams import DataStream
from fuel.datasets import IterableDataset
data_stream = DataStream(IterableDataset(
  numpy.random.rand(150).astype(theano.config.floatX)
))

Now let’s train with gradient descent and plot the results.

In [ ]:
from blocks.main_loop import MainLoop
from blocks.algorithms import GradientDescent, Scale
from blocks.extensions import FinishAfter
from blocks.extensions.monitoring import TrainingDataMonitoring
from blocks.extras.extensions.plot import Plot  
main_loop = MainLoop(
     model=None, data_stream=data_stream,
     algorithm=GradientDescent(cost=cost,
                               params=[a],
                               step_rule=Scale(learning_rate=0.1)),
     extensions=[FinishAfter(after_n_epochs=1),
                 TrainingDataMonitoring([cost, a], after_batch=True),
                 Plot('Plotting example', channels=[['cost'], ['a']],
                      after_batch=True)])  
main_loop.run() 